# Job Dataset Exploration

In [1]:
import pandas as pd
import numpy as np
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.algorithms import user_knn
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser

In [2]:
jobs_data = pd.read_csv("dataset/splitjobs/jobs1.tsv", sep='\t', error_bad_lines=False)
jobs_df = jobs_data[["JobID", "Title"]]
jobs_df = jobs_df.rename(columns={"JobID": "item", "Title": "title"})

/var/folders/vt/47mxbcvj5rd7mmyk053h49m00000gn/T/ipykernel_71678/1825902803.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  jobs_data = pd.read_csv("dataset/splitjobs/jobs1.tsv", sep='\t', error_bad_lines=False)
b'Skipping line 122433: expected 11 fields, saw 12\n'
/var/folders/vt/47mxbcvj5rd7mmyk053h49m00000gn/T/ipykernel_71678/1825902803.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs_data = pd.read_csv("dataset/splitjobs/jobs1.tsv", sep='\t', error_bad_lines=False)


In [17]:
# read the data (about applications)
data_train = pd.read_csv('dataset/user_with_negative_rating_train.csv', delimiter=',')
data_full = pd.read_csv('dataset/user_with_negative_ratings_full.csv', delimiter=',')
data_test = pd.read_csv('dataset/user_with_negtive_rating_test.csv', delimiter=',')


data = pd.concat([data_train.drop("Unnamed: 0", axis=1), data_full.loc[10000:,:]])
# construct dataframe in format (user, item, rating) via column addition
#df_ui.insert(2, "rating", np.random.rand(df_ui.shape[0],1))  
df_ui = data.rename(columns={"UserID": "user", "JobID": "item", "Rating":"rating"})
display(df_ui.head(10)) # check data being read properly

,user,item,rating
0,47,284009,1
1,47,2121,1
2,47,848187,1
3,47,733748,1
4,47,576958,1
5,47,262470,1
6,47,602298,1
7,47,982331,0
8,47,979937,0
9,47,821993,0


In [43]:
# We use the collaborative user algorithm UserUser, that use the nearest neighbors 
user_user = UserUser(45, min_nbrs=6)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(df_ui)

In [44]:
user_id = 4022

selected_jobs = recsys.recommend(user_id, 100) # generate 10 recommendations for the selected user
# score - doesn't matter cause arbitrary value showing recommendation priority
# sometimes ipynb doesn't reset the variables -> gives stupid errors (solution shove everything into one kernel/ block)

selected_jobs = selected_jobs.join(jobs_df.set_index('item'), on='item')
display(selected_jobs.head(20))

# Show the real applications
apps_data = pd.read_csv("dataset/apps.tsv", sep='\t')
apps_data = apps_data.loc[apps_data['UserID'] == user_id]
apps_data = apps_data.rename(columns={"JobID": "item"})
display(apps_data.join(jobs_df.set_index('item'), on='item').head(1))




,item,score,title
0,1100673,1.151754,Medical Biller/Collector
1,741780,1.000559,Operator-Part Time Evenings(Fri-Sun)
2,522858,0.968708,Receptionist Needed ASAP!
3,1082690,0.940608,Marketing Assistant
4,611716,0.847704,Accounts Payable Clerk
5,659222,0.847704,Admissions Representative
6,530598,0.822992,Resident Services Coordinator
7,260468,0.742833,Customer Service Representative
8,208057,0.738994,Financial Admin Assistant


,UserID,WindowID,Split,ApplicationDate,item,title
641,4022,1,Train,2012-04-02 10:44:32.04,459306,Inside Sales


In [28]:
data_test = data_test.rename(columns={"JobID": "item"})
display(data_test[data_test['UserID']==user_id].join(jobs_df.set_index('item'), on='item'))

,Unnamed: 0,index,UserID,item,Rating,title
0,0,0,47,169528,1,"Resort Host/Marketing Coordinator - Anaheim, CA"
